<a href="https://colab.research.google.com/github/SwethaVipparla/Tensorflow/blob/master/Advanced%20Tensorflow%20Usage/Mirrored_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
# additional imports

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [3]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape", y_train.shape)

170500096/170498071 [==============================] - 6s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape (50000,)


In [4]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [5]:
# Build the model using the functional API
def create_model():
  i = Input(shape=x_train[0].shape)

  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model

In [6]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [7]:
print(f'Number of devices: {strategy.num_replicas_in_sync}')

Number of devices: 1


In [8]:
with strategy.scope():
  model = create_model()

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [9]:
# Fit
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.3161 - accuracy: 0.5500 - val_loss: 0.9930 - val_accuracy: 0.6465
Epoch 2/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8510 - accuracy: 0.7057 - val_loss: 1.0277 - val_accuracy: 0.6603
Epoch 3/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.6880 - accuracy: 0.7642 - val_loss: 0.6683 - val_accuracy: 0.7736
Epoch 4/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.5870 - accuracy: 0.7995 - val_loss: 0.8439 - val_accuracy: 0.7284
Epoch 5/5
1563/1563 [==============================] - 11s 7ms/step - loss: 0.5021 - accuracy: 0.8298 - val_loss: 0.6186 - val_accuracy: 0.7960


In [10]:
50000/391

127.8772378516624

In [11]:
10000/79

126.58227848101266

In [12]:
# Compare this to non-distributed training
model2 = create_model()
model2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
1563/1563 [==============================] - 9s 6ms/step - loss: 1.3173 - accuracy: 0.5475 - val_loss: 0.9921 - val_accuracy: 0.6608
Epoch 2/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8511 - accuracy: 0.7049 - val_loss: 0.7787 - val_accuracy: 0.7373
Epoch 3/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.6982 - accuracy: 0.7610 - val_loss: 0.8131 - val_accuracy: 0.7289
Epoch 4/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.5874 - accuracy: 0.7988 - val_loss: 0.8222 - val_accuracy: 0.7262
Epoch 5/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.5032 - accuracy: 0.8286 - val_loss: 0.7082 - val_accuracy: 0.7809
